In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)


c:\Users\server\Desktop\john_capstone\capstone\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\server\Desktop\john_capstone\capstone\myenv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
import sys
sys.path.append('../dataset')
import get_funsd


In [4]:
funsd = get_funsd.CustomFunsdDataset('../dataset/')
funsd.split_generators()

In [5]:
funsd

CustomFunsdDataset:
DatasetDict({
    train: Dataset({features: ['id', 'tokens', 'ner_boxes', 'bboxes', 'ner_tags','line_ids','linkings','image','image_name'], num_rows: 149}),
    test: Dataset({features: ['id', 'tokens', 'ner_boxes', 'bboxes', 'ner_tags','line_ids','linkings','image','image_name'], num_rows: 50})
})

In [6]:
label_map = {
    0: 'O',
    1: 'B-HEADER',
    2: 'I-HEADER',
    3: 'B-QUESTION',
    4: 'I-QUESTION',
    5: 'B-ANSWER', 
    6: 'I-ANSWER'}

In [7]:
funsd['train'][0].keys()

dict_keys(['id', 'tokens', 'ner_boxes', 'bboxes', 'ner_tags', 'line_ids', 'linkings', 'image', 'image_name'])

In [8]:
from datasets import Dataset, DatasetDict

funsd_train_dataset = Dataset.from_dict({
    "id": [entry["id"] for entry in funsd["train"]],
    "tokens": [entry["tokens"] for entry in funsd["train"]],
    "ner_boxes": [entry["ner_boxes"] for entry in funsd["train"]],
    "bboxes": [entry["bboxes"] for entry in funsd["train"]],
    "ner_tags": [entry["ner_tags"] for entry in funsd["train"]],
    "line_ids": [entry["line_ids"] for entry in funsd["train"]],
    "linkings": [entry["linkings"] for entry in funsd["train"]],
    "image": [entry["image"] for entry in funsd["train"]],
    "image_name": [entry["image_name"] for entry in funsd["train"]],
})

funsd_test_dataset = Dataset.from_dict({
    "id": [entry["id"] for entry in funsd["test"]],
    "tokens": [entry["tokens"] for entry in funsd["test"]],
    "ner_boxes": [entry["ner_boxes"] for entry in funsd["test"]],
    "bboxes": [entry["bboxes"] for entry in funsd["test"]],
    "ner_tags": [entry["ner_tags"] for entry in funsd["test"]],
    "line_ids": [entry["line_ids"] for entry in funsd["test"]],
    "linkings": [entry["linkings"] for entry in funsd["test"]],
    "image": [entry["image"] for entry in funsd["test"]],
    "image_name": [entry["image_name"] for entry in funsd["test"]],
})

# Optionally, you can create a DatasetDict if you have train/test splits
dataset = DatasetDict({
    "train": funsd_train_dataset,
    'test': funsd_test_dataset
})


In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_boxes', 'bboxes', 'ner_tags', 'line_ids', 'linkings', 'image', 'image_name'],
        num_rows: 149
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_boxes', 'bboxes', 'ner_tags', 'line_ids', 'linkings', 'image', 'image_name'],
        num_rows: 50
    })
})

In [10]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [11]:
import sys
sys.path.append('../..')
from mytools import unnormalize_box, normalize_box

In [12]:

import numpy as np
def make_dataset(documents):
    images = documents['image']
    
    rgb_images = list(map(lambda img: img.convert('RGB'), images))
  
    words = documents['tokens']
    
    boxes = documents['bboxes'] ## token boxes normalized below
    
    word_labels = documents['ner_tags']
    
    normalized_boxes = []

    # Normalize the boxes by image width and height
    for i in range(len(boxes)):
        normalized_boxes.append([])
        for j in range(len(boxes[i])):
            normalized_boxes[-1].append(normalize_box(boxes[i][j], images[i].width, images[i].height))

    encoding = processor(rgb_images, words, boxes=normalized_boxes, word_labels=word_labels,
                         truncation=True, padding="max_length", return_offsets_mapping=True)
    
    encoding.pop('offset_mapping')

    return encoding

In [13]:
cols = funsd_train_dataset.column_names



Standardization: Many popular image classification models, like AlexNet, ResNet, and Vision Transformers, originally used 224x224 as a default input size. It has since become a standard, simplifying the process of reusing pre-trained models and ensuring consistency across various tasks.

Computational Efficiency: A size of 224x224 provides a good balance between image detail and computational cost. If images were too large, the model would require significantly more memory and processing power, but if they were too small, important details could be lost. 224x224 strikes a balance where enough spatial information is preserved while keeping the model efficient.

Training and Pre-training: Many models, including LayoutLMv3, are pre-trained on large datasets like ImageNet, where images are typically resized to 224x224. To take advantage of this pre-training and transfer learning, using the same input size ensures compatibility with the pre-trained weights and architectures.

224x224 is automated by layoutlm3

Bounding boxes: Stay on a 0-1000 scale to maintain relative spatial positioning of text or entities in the document.
Images: Resized to 224x224 pixels to meet the input size requirement of the visual part of the model.

In [14]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

features = Features({
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(feature=Value(dtype='int64')),
})
train_dataset = dataset["train"].map(
    make_dataset,
    batched=True,
    remove_columns = cols,
    features=features)

Map: 100%|██████████| 149/149 [00:01<00:00, 75.65 examples/s]


In [15]:
test_dataset = dataset["test"].map(
    make_dataset,
    batched=True,
    remove_columns = cols,
    features=features)

Map: 100%|██████████| 50/50 [00:00<00:00, 78.86 examples/s]


In [16]:
print(processor.tokenizer.decode(train_dataset[0]["input_ids"]))

<s> R&D : Suggestion: Date: Licensee Yes No 597005708 R&D QUALITY IMPROVEMENT SUGGESTION/ SOLUTION FORM Name / Phone Ext. : M. Hamann P. Harper, P. Martinez 9/ 3/ 92 R&D Group: J. S. Wigand Supervisor / Manager Discontinue coal retention analyses on licensee submitted product samples (Note : Coal Retention testing is not performed by most licensees. Other B&W physical measurements as ends stability and inspection for soft spots in ciparettes are thought to be sufficient measures to assure cigarette physical integrity. The proposed action will increase laboratory productivity. ) Suggested Solutions (s) : Delete coal retention from the list of standard analyses performed on licensee submitted product samples. Special requests for coal retention testing could still be submitted on an exception basis. Have you contacted your Manager/ Supervisor? Manager Comments: Manager, please contact suggester and forward comments to the Quality Council. qip. wp</s><pad><pad><pad><pad><pad><pad><pad><pa

In [17]:
train_dataset.set_format("torch")
test_dataset.set_format("torch")

In [18]:
example = train_dataset[0]
for k,v in example.items():
    print(k,v.shape)


pixel_values torch.Size([3, 224, 224])
input_ids torch.Size([512])
attention_mask torch.Size([512])
bbox torch.Size([512, 4])
labels torch.Size([512])


In [19]:
c=0
for id, label in zip(train_dataset[0]["input_ids"], train_dataset[0]["labels"]):
  print(processor.tokenizer.decode([id]), label.item())
  c+=1
  if c>10:break

<s> -100
 R 0
& -100
D -100
 : 3
 Suggest 3
ion -100
: -100
 Date 3
: -100
 License 5


In [20]:
from evaluate import load 
metric = load("seqeval") 

In [21]:
label_list = ['O', 'B-HEADER', 'I-HEADER', 'B-QUESTION', 'I-QUESTION', 'B-ANSWER', 'I-ANSWER']

In [22]:
import numpy as np

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [23]:
label_map_reversed = {v: k for k, v in label_map.items()} 


In [24]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from transformers.data.data_collator import default_data_collator
from transformers.trainer_callback import TrainerCallback
import optuna
import os
from transformers import LayoutLMv3ForTokenClassification
import json
import numpy as np



class MetricsTrackingCallback(TrainerCallback):
    def __init__(self):
        self.metrics_history = {
            'train': [],
            'eval': [],
            'epoch': []
        }
        self.current_train_metrics = None
        self.last_logged_epoch = None

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """Called after evaluation"""
        if metrics and state.epoch != self.last_logged_epoch:
            # Store evaluation metrics
            eval_metrics = {
                'precision': metrics.get('eval_precision', None),
                'recall': metrics.get('eval_recall', None),
                'f1': metrics.get('eval_f1', None),
                'accuracy': metrics.get('eval_accuracy', None)
            }
            
            # Store training metrics that were saved during training
            train_metrics = {}
            if self.current_train_metrics is not None:
                train_metrics = self.current_train_metrics
                self.current_train_metrics = None  # Reset for next epoch
            
            # Append metrics for this epoch
            self.metrics_history['train'].append(train_metrics)
            self.metrics_history['eval'].append(eval_metrics)
            self.metrics_history['epoch'].append(state.epoch)
            self.last_logged_epoch = state.epoch
            
            # Save to file
            save_dir = args.output_dir
            os.makedirs(save_dir, exist_ok=True)
            with open(os.path.join(save_dir, 'metrics_history.json'), 'w') as f:
                json.dump(self.metrics_history, f, indent=2)

    def on_log(self, args, state, control, logs=None, **kwargs):
        """Called on each training log"""
        if logs:
            # Check if these are training metrics (not eval metrics)
            if all(not k.startswith('eval_') for k in logs.keys()):
                # Store the most recent training metrics
                self.current_train_metrics = {
                    'loss': logs.get('loss', None),
                    'learning_rate': logs.get('learning_rate', None)
                }

hyperparameter_dict = {}

def objective(trial):
    model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                             id2label=label_map,
                                                             label2id=label_map_reversed).to(device)
    
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-4, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True)
    batch_size = 2
    
    model.config.hidden_dropout_prob = dropout
    model.config.attention_probs_dropout_prob = dropout
    
    trial_output_dir = f'./results/'
    training_args = TrainingArguments(
        output_dir=trial_output_dir,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=200,
        weight_decay=weight_decay,
        logging_strategy="epoch",        # Log once per epoch
        logging_first_step=True,         # Log the first step
        evaluation_strategy="epoch",     # Evaluate once per epoch
        save_strategy="epoch",
        metric_for_best_model="eval_f1",
        load_best_model_at_end=True,
        save_total_limit=1
    )
    
    # Initialize callbacks
    metrics_callback = MetricsTrackingCallback()
    early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=5)
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset.select(range(119)),
        eval_dataset=train_dataset.select(range(119, len(train_dataset))),
        tokenizer=processor,
        data_collator=default_data_collator,
        compute_metrics=compute_metrics,
        callbacks=[early_stopping_callback, metrics_callback]
    )
    
    train_result = trainer.train()
    eval_results = trainer.evaluate()
    
    # Store all metrics including per-epoch history
    hyperparameter_dict[trial.number] = {
        "final_eval_results": eval_results,
        "metrics_history": metrics_callback.metrics_history,
        "hyperparameters": {
            "dropout": dropout,
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "batch_size": batch_size,
        }
    }
    
    return eval_results["eval_f1"]

In [25]:
# Run the optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

[I 2024-10-21 13:46:22,096] A new study created in memory with name: no-name-98ddbe1d-0de9-41be-a3a7-d9dece3434ac
Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\server\Desktop\john_capstone\capstone\myenv\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/12000 [00:00<?, ?it/s]c:\Users\server\Desktop\john_capstone\capstone\myenv\lib\site-packages\transformers\modeling_utils.py:1126: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
  0%|          | 2/12000 [00:00<1:03:52,  3.13it/s]

{'loss': 1.8252, 'grad_norm': 13.041524887084961, 'learning_rate': 0.0001553399326958385, 'epoch': 0.02}


  0%|          | 60/12000 [00:11<33:21,  5.96it/s] 

{'loss': 1.7925, 'grad_norm': 2.181394338607788, 'learning_rate': 0.0001545761143752239, 'epoch': 1.0}


c:\Users\server\Desktop\john_capstone\capstone\myenv\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                  
  0%|          | 60/12000 [00:12<33:21,  5.96it/s]

...


  6%|▌         | 720/12000 [02:26<26:41,  7.04it/s]

{'eval_loss': 0.9992364645004272, 'eval_precision': 0.7292768959435626, 'eval_recall': 0.7504537205081669, 'eval_f1': 0.7397137745974954, 'eval_accuracy': 0.7891440501043842, 'eval_runtime': 0.8237, 'eval_samples_per_second': 36.421, 'eval_steps_per_second': 18.21, 'epoch': 12.0}


  6%|▌         | 720/12000 [02:28<38:53,  4.83it/s]
c:\Users\server\Desktop\john_capstone\capstone\myenv\lib\site-packages\transformers\modeling_utils.py:1126: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


{'train_runtime': 148.8503, 'train_samples_per_second': 159.892, 'train_steps_per_second': 80.618, 'train_loss': 0.354268714123302, 'epoch': 12.0}


100%|██████████| 15/15 [00:00<00:00, 19.53it/s]
[I 2024-10-21 14:46:33,792] Trial 19 finished with value: 0.7860608734009703 and parameters: {'dropout': 0.37239684504038983, 'learning_rate': 3.2446935027014366e-05, 'weight_decay': 0.0006974053458588097}. Best is trial 18 with value: 0.8112133158125274.


In [26]:
import pickle, json

# Save the study to a pickle file
with open("optuna_study.pkl", "wb") as f:
    pickle.dump(study, f)


with open("tuning.json", "w") as f:
    json.dump(hyperparameter_dict, f, indent=4)

In [27]:
import pickle, json
with open("optuna_study.pkl", "rb") as f:
    study = pickle.load(f)

with open("tuning.json", "r") as f:
    hyperparameter_dict = json.load(f)

In [33]:
study.best_params, study.best_value , study.best_trial

({'dropout': 0.3560592494053517,
  'learning_rate': 3.127610016621582e-05,
  'weight_decay': 0.000817139913231531},
 0.8112133158125274,
 FrozenTrial(number=18, state=TrialState.COMPLETE, values=[0.8112133158125274], datetime_start=datetime.datetime(2024, 10, 21, 14, 40, 26, 227396), datetime_complete=datetime.datetime(2024, 10, 21, 14, 44, 3, 193969), params={'dropout': 0.3560592494053517, 'learning_rate': 3.127610016621582e-05, 'weight_decay': 0.000817139913231531}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'dropout': FloatDistribution(high=0.5, log=False, low=0.1, step=None), 'learning_rate': FloatDistribution(high=0.0005, log=True, low=1e-05, step=None), 'weight_decay': FloatDistribution(high=0.01, log=True, low=1e-06, step=None)}, trial_id=18, value=None))

In [34]:
hyperparameter_dict['18']

{'final_eval_results': {'eval_loss': 0.9641380906105042,
  'eval_precision': 0.7840812870448772,
  'eval_recall': 0.8402903811252269,
  'eval_f1': 0.8112133158125274,
  'eval_accuracy': 0.800365344467641,
  'eval_runtime': 0.8488,
  'eval_samples_per_second': 35.346,
  'eval_steps_per_second': 17.673,
  'epoch': 16.0},
 'metrics_history': {'train': [{'loss': 1.2479,
    'learning_rate': 3.111971966538474e-05},
   {'loss': 0.7267, 'learning_rate': 3.096333916455366e-05},
   {'loss': 0.5519, 'learning_rate': 3.080695866372258e-05},
   {'loss': 0.4494, 'learning_rate': 3.0650578162891504e-05},
   {'loss': 0.3332, 'learning_rate': 3.0494197662060425e-05},
   {'loss': 0.2696, 'learning_rate': 3.0337817161229346e-05},
   {'loss': 0.1908, 'learning_rate': 3.0181436660398264e-05},
   {'loss': 0.1589, 'learning_rate': 3.0025056159567185e-05},
   {'loss': 0.124, 'learning_rate': 2.9868675658736106e-05},
   {'loss': 0.121, 'learning_rate': 2.9712295157905027e-05},
   {'loss': 0.0753, 'learning_ra

In [ ]:
print(f"Best trial: \nValue: {study.best_trial.value}\nParams: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
study.best_params

In [9]:
dropout = study.best_params["dropout"]
learning_rate = study.best_params["learning_rate"]
weight_decay = study.best_params["weight_decay"]

In [30]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(".\laoutlm3_output/checkpoint-100")

In [33]:
def getf1(example):
    image = example["image"].convert("RGB")
    words = example["tokens"]
    boxes = example["bboxes"]
    normalized_boxes = []

    for i in range(len(boxes)):
        normalized_boxes.append(normalize_box(boxes[i], image.width, image.height))


    normalized_boxes = torch.tensor(normalized_boxes, dtype=torch.long)
    
    word_labels = example["ner_tags"]
    word_labels = torch.tensor(word_labels, dtype=torch.long)

    
    encoding = processor(image, words, boxes=normalized_boxes, word_labels=word_labels,
                         truncation=True, padding="max_length", return_tensors="pt")
    
    with torch.no_grad():
        outputs = model(**encoding)
        
    logits = outputs.logits
    predictions = logits.argmax(-1).squeeze().tolist()
    labels = encoding.labels.squeeze().tolist()
    
    true_predictions = [model.config.id2label[pred] for pred, label in zip(predictions, labels) if label != - 100]
    true_labels = [model.config.id2label[label] for prediction, label in zip(predictions, labels) if label != -100]

    correct_predictions = sum(t == p for t, p in zip(true_labels, true_predictions))
    all_tokens = len(true_labels)
    

    return correct_predictions, all_tokens


In [ ]:
correct = 0
all_tokens = 0
for i in range(len(funsd_test_dataset)):
    ci,ai = getf1(funsd_test_dataset[i])
    correct += ci
    all_tokens += ai


In [ ]:
correct/all_tokens

In [ ]:
from PIL import ImageDraw, ImageFont

draw = ImageDraw.Draw(image)

font = ImageFont.load_default()

def iob_to_label(label):
    label = label[2:]
    if not label:
      return 'other'
    return label

label2color = {'question':'blue', 'answer':'green', 'header':'orange', 'other':'violet'}

for prediction, box in zip(true_predictions,true_boxes ):
    predicted_label = iob_to_label(prediction).lower()
    draw.rectangle(box, outline=label2color[predicted_label])
    draw.text((box[0] + 10, box[1] - 10), text=predicted_label, fill=label2color[predicted_label], font=font)

image

In [ ]:
image = example["image"]
image = image.convert("RGB")

draw = ImageDraw.Draw(image)

for word, box, label in zip(example['tokens'], example['bboxes'], example['ner_tags']):
  actual_label = iob_to_label(label_map[label]).lower()
  draw.rectangle(box, outline=label2color[actual_label], width=2)
  draw.text((box[0] + 10, box[1] - 10), actual_label, fill=label2color[actual_label], font=font)

image

In [ ]:
### put an image from epant !!!!!!!!!!!!!!!!!!!!!!!!!!!!!